In [ ]:
import setup

setup.init()

In [ ]:
from reddit.models import RedditCommunity

import ai

In [ ]:
ai.extract_topics_agent(
    "Why are there cameras everywhere? Wouldn't the world be better without cameras?"
)

In [ ]:
query = "I really like snowboarding"

topics = ai.extract_topics_agent(query)
topic_names = ", ".join([x.get("name") for x in topics])

topic_names

In [ ]:
for topic in topics:
    topic_name = topic.get("name")

    print(f"Researching {topic_name}")

    community_data = ai.perform_get_reddit_communities(topic_name)

    for community in community_data:
        url = community.pop("url")

        print(community)

        RedditCommunity.objects.update_or_create(url=url, defaults=community)